# GCN practice code

- import basic library

In [1]:
import torch

torch.version

<module 'torch.version' from 'e:\\conda3\\envs\\test\\lib\\site-packages\\torch\\version.py'>

In [2]:
import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx

import networkx as nx
import numpy as np
from random import randint, expovariate
import matplotlib.pyplot as plt

- Generate the network

In [3]:
S_CPU_MAX = []
S_BW_MAX = []

# 随机生成一个图(20节点, 100链路)
net = nx.gnm_random_graph(n=20, m=100)

# 设置所有节点的CPU数据, 并同时统计最大值最小值
min_cpu_capacity = 1.0e10
max_cpu_capacity = 0.0
for node_id in net.nodes:
    net.nodes[node_id]['CPU'] = randint(50, 100)
    net.nodes[node_id]['LOCATION'] = randint(0, 2)
    
    if net.nodes[node_id]['CPU'] < min_cpu_capacity:
        min_cpu_capacity = net.nodes[node_id]['CPU']
    if net.nodes[node_id]['CPU'] > max_cpu_capacity:
        max_cpu_capacity = net.nodes[node_id]['CPU']

# 设置链路的带宽数据, 并同时统计最大带宽最小带宽
min_bandwidth_capacity = 1.0e10
max_bandwidth_capacity = 0.0
for edge_id in net.edges:
    net.edges[edge_id]['bandwidth'] = randint(50, 100)
    
    if net.edges[edge_id]['bandwidth'] < min_bandwidth_capacity:
        min_bandwidth_capacity = net.edges[edge_id]['bandwidth']
    if net.edges[edge_id]['bandwidth'] > max_bandwidth_capacity:
        max_bandwidth_capacity = net.edges[edge_id]['bandwidth']

# data=True: 返回的是 NodeDataView 对象, 该对象不仅包含每个顶点的 ID 属性, 还包括顶点的其他属性
for s_node_id, s_node_data in net.nodes(data=True):
    S_CPU_MAX.append(s_node_data['CPU'])

# 统计每个底层节点周围链路带宽和
for s_node_id in range(len(net.nodes)):
    total_node_bandwidth = 0.0
    for link_id in net[s_node_id]:
        total_node_bandwidth += net[s_node_id][link_id]['bandwidth']
    S_BW_MAX.append(total_node_bandwidth)

In [4]:
 # S_CPU_Free
s_CPU_remaining = []
s_bandwidth_remaining = []

# 1 表示目前哪些节点被占用, 0 相反
current_embedding = [0] * len(net.nodes)

# 节点剩余资源
for s_node_id, s_node_data in net.nodes(data=True):
    s_CPU_remaining.append(s_node_data['CPU'])
    
# 节点周围剩余带宽资源
for s_node_id in range(len(net.nodes)):
    total_node_bandwidth = 0.0
    for link_id in net[s_node_id]:
        total_node_bandwidth += net[s_node_id][link_id]['bandwidth']
    s_bandwidth_remaining.append(total_node_bandwidth)

In [5]:
# 底层网络特征矩阵
substrate_features = []
substrate_features.append(S_CPU_MAX)
substrate_features.append(S_BW_MAX)
substrate_features.append(s_CPU_remaining)
substrate_features.append(s_bandwidth_remaining)
substrate_features.append(current_embedding)

print(substrate_features)

[[74, 63, 78, 68, 90, 60, 95, 85, 72, 99, 51, 67, 58, 61, 69, 54, 71, 69, 88, 88], [720.0, 869.0, 933.0, 922.0, 823.0, 589.0, 689.0, 768.0, 577.0, 581.0, 723.0, 787.0, 970.0, 553.0, 828.0, 649.0, 608.0, 567.0, 743.0, 891.0], [74, 63, 78, 68, 90, 60, 95, 85, 72, 99, 51, 67, 58, 61, 69, 54, 71, 69, 88, 88], [720.0, 869.0, 933.0, 922.0, 823.0, 589.0, 689.0, 768.0, 577.0, 581.0, 723.0, 787.0, 970.0, 553.0, 828.0, 649.0, 608.0, 567.0, 743.0, 891.0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [6]:
substrate_features = torch.tensor(substrate_features)
print(substrate_features)
print(substrate_features.shape)

# transpose: 转置
substrate_features = torch.transpose(substrate_features, 0, 1)
print(substrate_features)
print(substrate_features.shape)

tensor([[ 74.,  63.,  78.,  68.,  90.,  60.,  95.,  85.,  72.,  99.,  51.,  67.,
          58.,  61.,  69.,  54.,  71.,  69.,  88.,  88.],
        [720., 869., 933., 922., 823., 589., 689., 768., 577., 581., 723., 787.,
         970., 553., 828., 649., 608., 567., 743., 891.],
        [ 74.,  63.,  78.,  68.,  90.,  60.,  95.,  85.,  72.,  99.,  51.,  67.,
          58.,  61.,  69.,  54.,  71.,  69.,  88.,  88.],
        [720., 869., 933., 922., 823., 589., 689., 768., 577., 581., 723., 787.,
         970., 553., 828., 649., 608., 567., 743., 891.],
        [  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
           0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.]])
torch.Size([5, 20])
tensor([[ 74., 720.,  74., 720.,   0.],
        [ 63., 869.,  63., 869.,   0.],
        [ 78., 933.,  78., 933.,   0.],
        [ 68., 922.,  68., 922.,   0.],
        [ 90., 823.,  90., 823.,   0.],
        [ 60., 589.,  60., 589.,   0.],
        [ 95., 689.,  95., 689.,   0.],
    

In [7]:
# substrate_features = torch.reshape(substrate_features, (-1,))
# print(substrate_features.shape)
# print(substrate_features)

In [8]:
# vnr_cpu = torch.tensor([10])
# vnr_bw = torch.tensor([30])
# pending = torch.tensor([2])
# substrate_features = torch.cat((substrate_features, vnr_cpu, vnr_bw, pending), 0)

# substrate_features
# substrate_features.shape

In [7]:
print(substrate_features)

tensor([[ 74., 720.,  74., 720.,   0.],
        [ 63., 869.,  63., 869.,   0.],
        [ 78., 933.,  78., 933.,   0.],
        [ 68., 922.,  68., 922.,   0.],
        [ 90., 823.,  90., 823.,   0.],
        [ 60., 589.,  60., 589.,   0.],
        [ 95., 689.,  95., 689.,   0.],
        [ 85., 768.,  85., 768.,   0.],
        [ 72., 577.,  72., 577.,   0.],
        [ 99., 581.,  99., 581.,   0.],
        [ 51., 723.,  51., 723.,   0.],
        [ 67., 787.,  67., 787.,   0.],
        [ 58., 970.,  58., 970.,   0.],
        [ 61., 553.,  61., 553.,   0.],
        [ 69., 828.,  69., 828.,   0.],
        [ 54., 649.,  54., 649.,   0.],
        [ 71., 608.,  71., 608.,   0.],
        [ 69., 567.,  69., 567.,   0.],
        [ 88., 743.,  88., 743.,   0.],
        [ 88., 891.,  88., 891.,   0.]])


- Using 'from_networkx'
    - transfer the torch_geometric

In [8]:
data = from_networkx(net)

In [9]:
print(data)

Data(CPU=[20], LOCATION=[20], bandwidth=[200], edge_index=[2, 200])


### Graph Convolution Network
- Generate the GCN class

In [10]:
from torch.nn import Linear
from torch_geometric.nn import GCNConv


class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        # in_channels: 节点特征数   out_channels: 输出的节点分类数
        self.conv1 = GCNConv(in_channels=5, out_channels=4)
        self.conv2 = GCNConv(in_channels=4, out_channels=4)
        self.conv3 = GCNConv(in_channels=4, out_channels=1)
        self.classifier = Linear(1, 20)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = h.tanh()
        h = self.conv2(h, edge_index)
        h = h.tanh()
        h = self.conv3(h, edge_index)
        h = h.tanh()  # Final GNN embedding space.
        
        # Apply a final (linear) classifier.
        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(5, 4)
  (conv2): GCNConv(4, 4)
  (conv3): GCNConv(4, 1)
  (classifier): Linear(in_features=1, out_features=20, bias=True)
)


In [11]:
model = GCN()

print(substrate_features.shape, data.edge_index.shape)
print(substrate_features)

out, embedding = model(substrate_features, data.edge_index)
# out, embedding = model(data.x, data.edge_index)
print(f'Embedding shape: {list(embedding.shape)}')

torch.Size([20, 5]) torch.Size([2, 200])
tensor([[ 74., 720.,  74., 720.,   0.],
        [ 63., 869.,  63., 869.,   0.],
        [ 78., 933.,  78., 933.,   0.],
        [ 68., 922.,  68., 922.,   0.],
        [ 90., 823.,  90., 823.,   0.],
        [ 60., 589.,  60., 589.,   0.],
        [ 95., 689.,  95., 689.,   0.],
        [ 85., 768.,  85., 768.,   0.],
        [ 72., 577.,  72., 577.,   0.],
        [ 99., 581.,  99., 581.,   0.],
        [ 51., 723.,  51., 723.,   0.],
        [ 67., 787.,  67., 787.,   0.],
        [ 58., 970.,  58., 970.,   0.],
        [ 61., 553.,  61., 553.,   0.],
        [ 69., 828.,  69., 828.,   0.],
        [ 54., 649.,  54., 649.,   0.],
        [ 71., 608.,  71., 608.,   0.],
        [ 69., 567.,  69., 567.,   0.],
        [ 88., 743.,  88., 743.,   0.],
        [ 88., 891.,  88., 891.,   0.]])
Embedding shape: [20, 1]


In [12]:
print(embedding)
print(embedding.shape)

tensor([[0.9108],
        [0.9289],
        [0.9293],
        [0.9311],
        [0.9205],
        [0.8647],
        [0.9087],
        [0.9105],
        [0.8877],
        [0.8866],
        [0.9094],
        [0.9258],
        [0.9394],
        [0.8805],
        [0.9247],
        [0.9013],
        [0.8805],
        [0.8671],
        [0.9199],
        [0.9296]], grad_fn=<TanhBackward0>)
torch.Size([20, 1])


In [13]:
print(out.shape)
print(out)

torch.Size([20, 20])
tensor([[ 0.6989, -0.1643,  1.0958,  0.2376,  0.0800, -0.7598, -0.6396,  0.2236,
          0.8639,  0.2120,  0.9899,  0.2930,  0.2806,  0.1531, -0.7991,  1.3409,
         -0.8042,  1.2360,  0.3351, -1.1074],
        [ 0.6979, -0.1681,  1.1019,  0.2300,  0.0874, -0.7710, -0.6450,  0.2374,
          0.8657,  0.1978,  1.0007,  0.2811,  0.2795,  0.1531, -0.8071,  1.3489,
         -0.8060,  1.2459,  0.3442, -1.1215],
        [ 0.6979, -0.1681,  1.1020,  0.2298,  0.0876, -0.7712, -0.6452,  0.2378,
          0.8657,  0.1974,  1.0010,  0.2808,  0.2794,  0.1531, -0.8073,  1.3491,
         -0.8060,  1.2461,  0.3445, -1.1219],
        [ 0.6978, -0.1685,  1.1027,  0.2291,  0.0883, -0.7724, -0.6457,  0.2392,
          0.8659,  0.1960,  1.0021,  0.2796,  0.2793,  0.1531, -0.8080,  1.3500,
         -0.8062,  1.2471,  0.3454, -1.1233],
        [ 0.6984, -0.1663,  1.0991,  0.2335,  0.0840, -0.7658, -0.6425,  0.2310,
          0.8649,  0.2044,  0.9957,  0.2866,  0.2800,  0.1531, -0.

# A3C Code
- Simple A3C code

In [14]:
from torch import nn
import torch
import numpy as np


def v_wrap(np_array, dtype=np.float32):
    """
    将numpy数据转换为torch类型
    """
    if np_array.dtype != dtype:
        np_array = np_array.astype(dtype)
    return torch.from_numpy(np_array)


def set_init(layers):
    """
    所有网络层初始化
    """
    for layer in layers:
        nn.init.normal_(layer.weight, mean=0., std=0.1)
        nn.init.constant_(layer.bias, 0.)


def push_and_pull(opt, lnet, gnet, done, s_, bs, buffer_action, buffer_reward, gamma):
    """
    pull: 把主网络的参数直接赋予Worker中的网络
    push: 使用各Worker中的梯度, 对主网络的参数进行更新
    """
    if done:
        v_s_ = 0.               # terminal
    else:
        v_s_ = lnet.forward(v_wrap(s_[None, :]))[-1].data.numpy()[0, 0]

    buffer_v_target = []
    for r in buffer_reward[::-1]:    # reverse buffer r
        v_s_ = r + gamma * v_s_
        buffer_v_target.append(v_s_)
    buffer_v_target.reverse()
    
    
    loss = lnet.loss_func(
        v_wrap(np.vstack(bs)),
        v_wrap(np.array(buffer_action), dtype=np.int64) if buffer_action[0].dtype == np.int64 else v_wrap(np.vstack(buffer_action)),
        v_wrap(np.array(buffer_v_target)[:, None]))

    # calculate local gradients and push local parameters to global
    opt.zero_grad()
    loss.backward()
    for lp, gp in zip(lnet.parameters(), gnet.parameters()):
        gp._grad = lp.grad
    opt.step()

    # pull global parameters
    lnet.load_state_dict(gnet.state_dict())


def record(global_ep, global_ep_r, ep_r, res_queue, name):
    """
    
    """
    with global_ep.get_lock():
        global_ep.value += 1
        
    with global_ep_r.get_lock():
        if global_ep_r.value == 0.:
            global_ep_r.value = ep_r
        else:
            global_ep_r.value = global_ep_r.value * 0.99 + ep_r * 0.01
            
    res_queue.put(global_ep_r.value)
    
    print(
        name,
        "Ep:", global_ep.value,
        "| Ep_r: %.0f" % global_ep_r.value,
    )

In [15]:
class SharedAdam(torch.optim.Adam):
    """
    优化器
    """
    def __init__(self, params, lr=1e-3, betas=(0.9, 0.99), eps=1e-8,
                 weight_decay=0):
        super(SharedAdam, self).__init__(params, lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        
        # State initialization
        for group in self.param_groups:
            for p in group['params']:
                state = self.state[p]
                state['step'] = 0
                state['exp_avg'] = torch.zeros_like(p.data)
                state['exp_avg_sq'] = torch.zeros_like(p.data)

                # share in memory
                state['exp_avg'].share_memory_()
                state['exp_avg_sq'].share_memory_()

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.multiprocessing as mp
import gym
import os
os.environ["OMP_NUM_THREADS"] = "1"

UPDATE_GLOBAL_ITER = 5
GAMMA = 0.9
MAX_EP = 3000

env = gym.make('CartPole-v1')
N_S = env.observation_space.shape[0]
N_A = env.action_space.n

print(N_S, N_A)

4 2


In [18]:
class Net(nn.Module):
    def __init__(self, s_dim, a_dim):
        super(Net, self).__init__()
        self.s_dim = s_dim
        self.a_dim = a_dim
        self.pi1 = nn.Linear(s_dim, 128)
        self.pi2 = nn.Linear(128, a_dim)
        self.v1 = nn.Linear(s_dim, 128)
        self.v2 = nn.Linear(128, 1)
        set_init([self.pi1, self.pi2, self.v1, self.v2])
        self.distribution = torch.distributions.Categorical

    def forward(self, x):
        """
        前向传播
        """
        pi1 = torch.tanh(self.pi1(x))
        logits = self.pi2(pi1)
        v1 = torch.tanh(self.v1(x))
        values = self.v2(v1)
        return logits, values

    def choose_action(self, s):
        """
        
        """
        self.eval()
        
        # 生成每个动作的概率, 选择最大的
        logits, _ = self.forward(s)
        prob = F.softmax(logits, dim=1).data
        m = self.distribution(prob)
        
        return m.sample().numpy()[0]

    def loss_func(self, s, a, v_t):
        """
        计算误差
        """
        self.train()
        
        logits, values = self.forward(s)
        
        # 误差
        td = v_t - values
        c_loss = td.pow(2)
        
        probs = F.softmax(logits, dim=1)
        m = self.distribution(probs)
        exp_v = m.log_prob(a) * td.detach().squeeze()
        a_loss = -exp_v
        total_loss = (c_loss + a_loss).mean()
        return total_loss

In [19]:
class Worker(mp.Process):
    ''
    
    def __init__(self, gnet, opt, global_ep, global_ep_r, res_queue, name):
        super(Worker, self).__init__()
        self.name = 'w%02i' % name
        self.g_ep, self.g_ep_r, self.res_queue = global_ep, global_ep_r, res_queue
        self.gnet, self.opt = gnet, opt
        self.lnet = Net(N_S, N_A)           # local network
        self.env = gym.make('CartPole-v0').unwrapped

    def run(self):
        """
        
        """
        total_step = 1
        while self.g_ep.value < MAX_EP:
            # 重置环境
            s = self.env.reset()
            
            # 记录
            buffer_s, buffer_action, buffer_reward = [], [], []
            
            # 总收益
            ep_r = 0.
            
            while True:
#                 if self.name == 'w00':
#                     self.env.render()
                print(s)
                
                # 根据当前环境, 生成一个动作action
                a = self.lnet.choose_action(v_wrap(s[None, :]))
                print(s[None, :])
                print(v_wrap(s[None, :]))
                
                # 执行action, 返回: 下一状态, reward, 是否结束, 其他信息
                s_, r, done, _ = self.env.step(a)
                if done: r = -1
                
                # 总收益
                ep_r += r
                
                # 记录
                buffer_action.append(a)
                buffer_s.append(s)
                buffer_reward.append(r)

                if total_step % UPDATE_GLOBAL_ITER == 0 or done:  # update global and assign to local net
                    # sync
                    push_and_pull(self.opt, self.lnet, self.gnet, done, s_, buffer_s, buffer_action, buffer_reward, GAMMA)
                    buffer_s, buffer_action, buffer_reward = [], [], []

                    if done:  # done and print information
                        record(self.g_ep, self.g_ep_r, ep_r, self.res_queue, self.name)
                        break
                s = s_
                total_step += 1
        self.res_queue.put(None)

In [20]:
gnet = Net(N_S, N_A)        # global network
gnet.share_memory()         # share the global parameters in multiprocessing
opt = SharedAdam(gnet.parameters(), lr=1e-4, betas=(0.92, 0.999))      # global optimizer

global_ep, global_ep_r, res_queue = mp.Value('i', 0), mp.Value('d', 0.), mp.Queue()
print(mp.cpu_count())

# parallel training
workers = [Worker(gnet, opt, global_ep, global_ep_r, res_queue, i) for i in range(mp.cpu_count())]
[w.start() for w in workers]
res = []                    # record episode reward to plot
while True:
    r = res_queue.get()
    if r is not None:
        res.append(r)
    else:
        break
[w.join() for w in workers]

import matplotlib.pyplot as plt
plt.plot(res)
plt.ylabel('Moving average ep reward')
plt.xlabel('Step')
plt.show()

12
